In [29]:
import re
import requests
from datetime import datetime
import locale
locale.setlocale(locale.LC_TIME, 'ja_JP.UTF-8')

def get_http_from_url(url):
    """指定されたURLからデータを取得する"""
    response = requests.get(url)
    response.encoding = "sjis"  # 文字コードを明示的に指定する
    response.raise_for_status()  # ステータスコードが200以外の場合に例外を発生させる
    return response.text

#kako_tuho_url= "https://sannan.nl/unsaku/dat/1676674457.dat"
ima_tuho_url = "https://sannan.nl/unsaku/dat/1684471634.dat"

data = get_http_from_url(ima_tuho_url)
pattern = r'(.*?) (</b>\((.*?)\)<b></b>) (\((.*?)\)<b>)<><>(.*?) (.*?) (ID:(.+))<>(.+) <>'
matches = re.findall(pattern, data)
result = []

# 行ごとのデータを格納するリスト
lines = data.split('\n')

# 日付をdatetime型にするためのフォーマット
format_string = '%Y/%m/%d(%a) %H:%M:%S.%f'

for line_num, line in enumerate(lines, start=1):
    match = re.match(pattern, line)
    if match:
        groups = match.groups()
        date_time = groups[5] + " " + groups[6]
        ninjaid, wacchoi = (groups[2].strip(" 認").replace("&#9989;", ""), groups[4])
        ninjaid = re.sub(r".*?(L\d+-)", "", ninjaid)
        j = re.split(r'【該当レス】 <br> |【違反部分】 <br> |【詳細・備考】 <br> ', groups[9])
        if len(j) > 1:
            comment = [x.strip().replace('<br>', '') for x in j if x.strip()]
            if len(comment) == 3:
                target_res, gaitou_rule, detail = comment
                detail = detail.replace("（このレスは通報フォームから送信されました）", "").strip()
                target_res = target_res.strip()
                gaitou_rule = gaitou_rule.strip()
            else:
                detail = comment

            result.append({
                "res_num": line_num,
                "date_time": datetime.strptime(date_time, format_string),
                "ninjaid": ninjaid,
                "wacchoi": wacchoi,
                "target_res": target_res,
                "gaitou_rule": gaitou_rule,
                "detail": detail
            })

(len(result))

624

In [51]:
admin_texts='''
管理モメン ★<><>2023/05/19(金) 13:47:14.62 ID:ADMIN<> 前スレ <br> https://sannan.nl/test/read.cgi/unsaku/1676674457/l50 <br>  <br> 議論や異論は要望・雑談スレでお願いします 。（通報スレで通報以外の書き込みをした場合、荒らしとして処分される可能性があります。) <br> https://sannan.nl/test/read.cgi/unsaku/1684471379/l50 <br>  <br> 通報する人へ <br>  <br> ・GRまたはLRのどの部分に違反しているか必ず書いてください。 <br> ・該当レスまたはスレのURLを書いてください。 <br>  <br> GRはトップページに記載してあります。 <br> https://sannan.nl/ <br>  <br> 依頼テンプレ↓ <br>  <br> 【該当レスまたはスレッド】 <br> ここに違反レスまたはスレのURLを書く <br>  <br> 【違反理由】 <br> ここにGRまたはLRのどの部分に違反しているかを書く <br> 例： GR1.スレッド・板の趣旨に無関係な投稿 <br>  <br> 【詳細・備考】 <hr><font color='red'>※tlv=2</font><hr><font color='red'>※追記 2023/06/01(木) 08:49:34</font> <br>  <br> なるべく通報フォームからの通報でお願いします <hr><font color='red'>※デフォ774→ !absnz </font><hr><font color='red'>※追記 2023/06/11(日) 03:25:56</font> <br>  <br> 被通報スレで通報されたことを話題にするようなレスを行った場合、GR1違反として処罰される可能性があります <hr><font color='red'>※追記 2023/06/11(日) 03:29:47</font> <br>  <br> 虚偽通報（確認不足など悪意がなくても）は処罰します <br> 通報する前に必ずスレをきちんと確認してください <hr><font color='red'>※追記 2023/06/11(日) 03:36:43</font> <br>  <br> ネタ通報は自重してください <br> 虚偽通報扱いされる可能性があります <hr><font color='red'>※スレタイ変更 2023/06/13(火) 12:27:35</font> <br> 変更前： 通報スレ★3 [&#x1f337;] [管理モメン★]<>通報スレ★3 [管理モメン★]
管理モメン ★<><>2023/05/19(金) 13:48:03.47 ID:ADMIN<> !tlv2 <>
管理モメン ★<><>2023/05/19(金) 13:57:13.84 ID:ADMIN<> &gt;&gt;3 <br> 今回はレベル半分にするだけにしときます <>
管理モメン ★<><>2023/05/19(金) 14:14:59.66 ID:ADMIN<> &gt;&gt;6 <br> banしました <>
管理モメン ★<><>2023/05/19(金) 14:17:52.22 ID:ADMIN<> &gt;&gt;8 <br> 対応しました <>
ぷにぷに ★<><>2023/05/19(金) 14:23:17.46 ID:PUNIPUNI<> &gt;&gt;10 <br> 規制&#128582; <>
管理モメン ★<><>2023/05/19(金) 14:23:42.48 ID:ADMIN<> &gt;&gt;11 <br> 対応しました <>
管理モメン ★<><>2023/05/19(金) 14:25:00.51 ID:ADMIN<> &gt;&gt;14 <br> 対応しました <>
管理モメン ★<><>2023/05/19(金) 14:25:27.46 ID:ADMIN<> &gt;&gt;15 <br> 既に消されてますよ <br> スレ再取得してみてください <>
管理モメン ★<><>2023/05/19(金) 14:31:30.76 ID:ADMIN<> &gt;&gt;20 <br> 対応しました <>
管理モメン ★<><>2023/05/19(金) 14:37:53.00 ID:ADMIN<> ここまで対応しました <>
管理モメン ★<><>2023/05/19(金) 19:23:54.97 ID:ADMIN<> &gt;&gt;27 <br> 安倍晋三15日間にしました <>
ぷにぷに ★<><>2023/05/20(土) 11:52:52.52 ID:PUNIPUNI<> &gt;&gt;31 <br> &gt;&gt;32 <br> 規制&#128582; <>
りんご ★<><>2023/05/20(土) 17:19:20.79 ID:ADMIN<> &gt;&gt;37 <br> 手元の確認したサイトによるとどうやらproxyやVPNではなかったようです <br> ただエミュレーター臭いのでまた怪しいものがあったら教えてください <>
りんご ★<><>2023/05/20(土) 17:33:52.19 ID:ADMIN<> &gt;&gt;38 <br> 次にFAXっぽい言動したら紛らわしいの強制コテつけると思います <>
'''

In [60]:
import re
import requests
from datetime import datetime
import locale
locale.setlocale(locale.LC_TIME, 'ja_JP.UTF-8')

def get_http_from_url(url):
    """指定されたURLからデータを取得する"""
    response = requests.get(url)
    response.encoding = "sjis"  # 文字コードを明示的に指定する
    response.raise_for_status()  # ステータスコードが200以外の場合に例外を発生させる
    return response.text

kako_tuho_url= "https://sannan.nl/unsaku/dat/1676674457.dat"
ima_tuho_url = "https://sannan.nl/unsaku/dat/1684471634.dat"

data = get_http_from_url(ima_tuho_url) + get_http_from_url(kako_tuho_url)
# 日付をdatetime型にするためのフォーマット
format_string = '%Y/%m/%d(%a) %H:%M:%S.%f'

pattern = r'^(.*?) ★<><>(\d{4}/\d{2}/\d{2}\(\w+\) \d{2}:\d{2}:\d{2}\.\d{2}) ID:(\w+)<> &gt;&gt;(\d+(?:-\d+)?(?:,\d+)?) <br> (.+) <>$'

lines = data.strip().split('\n')
results = []

for line in lines:
    match = re.match(pattern, line)
    if match:
        result = {
            "name": match.group(1),
            "date_time": datetime.strptime(match.group(2), format_string),
            "ID": match.group(3),
            "target_res": match.group(4),
            "saitei_detail": match.group(5)
        }
        results.append(result)

results


[{'name': '管理モメン',
  'date_time': datetime.datetime(2023, 5, 19, 13, 57, 13, 840000),
  'ID': 'ADMIN',
  'target_res': '3',
  'saitei_detail': '今回はレベル半分にするだけにしときます'},
 {'name': '管理モメン',
  'date_time': datetime.datetime(2023, 5, 19, 14, 14, 59, 660000),
  'ID': 'ADMIN',
  'target_res': '6',
  'saitei_detail': 'banしました'},
 {'name': '管理モメン',
  'date_time': datetime.datetime(2023, 5, 19, 14, 17, 52, 220000),
  'ID': 'ADMIN',
  'target_res': '8',
  'saitei_detail': '対応しました'},
 {'name': 'ぷにぷに',
  'date_time': datetime.datetime(2023, 5, 19, 14, 23, 17, 460000),
  'ID': 'PUNIPUNI',
  'target_res': '10',
  'saitei_detail': '規制&#128582;'},
 {'name': '管理モメン',
  'date_time': datetime.datetime(2023, 5, 19, 14, 23, 42, 480000),
  'ID': 'ADMIN',
  'target_res': '11',
  'saitei_detail': '対応しました'},
 {'name': '管理モメン',
  'date_time': datetime.datetime(2023, 5, 19, 14, 25, 0, 510000),
  'ID': 'ADMIN',
  'target_res': '14',
  'saitei_detail': '対応しました'},
 {'name': '管理モメン',
  'date_time': datetime.datetime(2

In [75]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib

# データフレームの作成
df = pd.DataFrame(results)

# データの集計
counts = df['name'].value_counts()

counts

管理モメン               264
流浪の民                 69
ころころ                 58
おぱんつ枢機卿              33
りんご                  32
ぷにぷに                 24
ウクレレちゃん&#129317;     19
!nocmd＠管理モメン          5
滑ら                    2
通報対応＠りんご              1
Name: name, dtype: int64